[View in Colaboratory](https://colab.research.google.com/github/battlerhythm/tensorflow/blob/master/4.3%20Deep%20NN.ipynb)

In [0]:
import tensorflow as tf
import numpy as np

# [털, 날개]
x_data = np.array([[0, 0], [1, 0], [1,1], [0, 0], [0, 0], [0, 1]])
y_data = np.array([
    [1, 0, 0], # 기타
    [0, 1, 0], # 포유류
    [0, 0, 1], # 조류
    [1, 0, 0],
    [1, 0, 0],
    [0, 0, 1]
])

In [0]:
X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

# 은닉층(Hidden layer)는 하이퍼파라미터이므로 실험을 통해 가장 적절한 수를 정하면 됩니다.
W1 = tf.Variable(tf.random_uniform([2, 10], -1., 1.))
W2 = tf.Variable(tf.random_uniform([10, 3], -1., 1.))
b1 = tf.Variable(tf.zeros([10]))
b2 = tf.Variable(tf.zeros([3]))

L1 = tf.add(tf.matmul(X, W1), b1)
L1 = tf.nn.relu(L1)

In [0]:
# 출력층
model = tf.add(tf.matmul(L1, W2), b2)

In [0]:
# Cost Function
# 텐서플로에서 제공하는 교차 엔트로피 함수를 사용합니다.
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=model))

# AdamOptimizer를 사용합니다.
# GradientDescentOptimizer보다 보편적으로 성능이 좋다고 알려져 있습니다.
optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
train_op = optimizer.minimize(cost)

In [5]:
# 텐서플로의 세션을 초기화합니다.
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

# 앞서 구성한 특징과 레이블 데이터를 이용해 학습을 100번 진행합니다.
for step in range(100):
    sess.run(train_op, feed_dict={X: x_data, Y: y_data})
    
    # 학습도중 10번에 한번씩 손실값을 출력해봅니다.
    if (step + 1) % 10 == 0:
        print(step + 1, sess.run(cost, feed_dict={X: x_data, Y: y_data}))

(10, 0.7920787)
(20, 0.57695264)
(30, 0.39431813)
(40, 0.26245376)
(50, 0.16978778)
(60, 0.108376496)
(70, 0.07128041)
(80, 0.049239676)
(90, 0.035706267)
(100, 0.027235804)


In [6]:
# 예측값인 model을 바로 출력하면 [0.2, 0.7, 0.1]과 같이 확률로 나오기 때문에,
# 요소 중 가장 큰 값의 인덱스를 찾아주는 argmax 함수를 사용하여 레이블 값을 출력하도록 했습니다.
prediction = tf.argmax(model, axis=1)
target = tf.argmax(Y, axis=1)
print('예측값: %s' % sess.run(prediction, feed_dict={X: x_data}))
print('실제값: %s' % sess.run(target, feed_dict={Y: y_data}))

예측값: [0 1 2 0 0 2]
실제값: [0 1 2 0 0 2]


In [7]:
is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도: %.2f' % sess.run(accuracy * 100, feed_dict={X: x_data, Y: y_data}))

sess.close()

정확도: 100.00
